Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# AML Pipelines with Data Dependency
In this notebook, we will see how we can build a pipeline with implicit data dependancy.

## Prerequisites and AML Basics
Make sure you go through the [00.configuration](../01.tutorials/00.configuration.ipynb) Notebook first if you haven't.

### AML and Pipeline SDK-specific Imports

In [ ]:
import azureml.core
from azureml.core import Workspace, Run, Experiment, Datastore
from azureml.core.compute import BatchAiCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute import DataFactoryCompute
from azureml.train.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData, StepSequence
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.steps import DataTransferStep
from azureml.pipeline.core import PublishedPipeline
from azureml.pipeline.core.graph import PipelineParameter

print("Pipeline SDK-specific imports completed")

### Initialize Workspace

Initialize a [workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace(class%29) object from persisted configuration.

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Default datastore (Azure file storage)
default_ds = ws.get_default_datastore() 
print("Default datastore's name: {}".format(default_ds.name))

def_blob_s = Datastore(ws, "workspaceblobstore")
print("Blobstore's name: {}".format(def_blob_s.name))

In [ ]:
# project folder
project_folder = './scripts'
    
print('Sample projects will be created in {}.'.format(project_folder))

### Required data and script files for the the tutorial
Sample files required to finish this tutorial are already copied to the project folder specified above. Even though the .py provided in the samples don't have much "ML work," as a data scientist, you will work on this extensively as part of your work. To complete this tutorial, the contents of these files are not very important. The one-line files are for demostration purpose only.

### Compute Targets
See the list of Compute Targets on the workspace.

In [ ]:
cts = ws.compute_targets
for ct in cts:
    print(ct)

#### Retrieve an already attached BatchAI cluster (if any)
[Batch AI](https://docs.microsoft.com/en-us/azure/batch-ai/overview) is a service for provisioning and managing clusters of Azure virtual machines for running machine learning workloads. Let's create a new Batch AI cluster in the current workspace, if it doesn't already exist. We will then run the training script on this compute target.

In [ ]:
# choose a name for your cluster
batchai_cluster_name = "batchai"

#Note that batch_ai is the name of the target object, not the compute target object
batch_ai = None
# see if this compute target already exists in the workspace
cts = ws.compute_targets
if not cts == None:
    for ct in cts:
        if (ct=="batchai"):
            batch_ai = ct
            break
            
if batch_ai == None:
    print('No compute target \'{}\' found. No worries, we will create one in the next call.'.format(batchai_cluster_name))
else:
    print(batch_ai)

#### Create and attach BatchAI cluster if one already doesn't exist
**This step will take about 3 minutes and is providing only sparse output in the process. Please make sure to wait until the call returns before moving to the next cell.**

In [ ]:
if batch_ai == None:
    print('creating a new BatchAI compute target...')
    provisioning_config = BatchAiCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                #vm_priority = 'lowpriority', # optional
                                                                autoscale_enabled = True,
                                                                cluster_min_nodes = 1, 
                                                                cluster_max_nodes = 4)

    print('creating a batchai cluster...')
    batch_ai = ComputeTarget.create(ws, batchai_cluster_name, provisioning_config)
        
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    print('wait for the minimum node count to become available...')
    batch_ai.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current BatchAI cluster status, use the 'status' property    
    print(batch_ai.status.serialize())
    
if not batch_ai == None:
    print("Skipping new compute creation because we already found BatchAI compute.")    

**Wait for this call to finish before proceeding (you will see the asterisk turning to a number).**

Now that you have created the compute target, let's see what the workspace's compute_targets() function returns. You should now see one entry named 'batchai' of type BatchAI.

#### Create and attach a DSVM cluster as a compute target
The example below shows creating/retrieving/attaching a DSVM. You can skip this step if you want to as we are not planning to use it as part of this tutorial.

**The below call may take up to 30 seconds.**

In [ ]:
# YOU MAY SKIP THIS BLOCK
from azureml.core.compute import DsvmCompute
# cpu dsvm
dsvm_name = "cpudsvm"
try:
    dsvm = DsvmCompute(ws, dsvm_name)
    print("found existing dsvm.")
except:
    print("creating new dsvm")
    dsvm_config = DsvmCompute.provisioning_configuration(vm_size="Standard_D2_v2")
    dsvm = DsvmCompute.create(ws, name=dsvm_name, provisioning_configuration=dsvm_config)
    dsvm.wait_for_completion(show_output=True)
print("DSVM atached")

**Wait for this call to finish before proceeding (you will see the asterisk turning to a number).**

## Building Pipeline Steps with Inputs and Outputs
As mentioned earlier, a step in the pipeline can take data as input. This data can be a data source that lives in one of the accessible data locations, or intermediate data produced by a previous step in the pipeline.

### Datasources
Datasource is represented by **[DataReference](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.data_reference.datareference?view=azure-ml-py)** object and points to data that lives in or is accessible from Datastore. DataReference could be a pointer to a file or a directory.

In [ ]:
# Reference the data uploaded to blob storage using DataReference
# Assign the datasource to blob_input_data variable

# DataReference(datastore, 
#               data_reference_name=None, 
#               path_on_datastore=None, 
#               mode='mount', 
#               path_on_compute=None, 
#               overwrite=False)

blob_input_data = DataReference(
    datastore=def_blob_s,
    data_reference_name="test_data",
    path_on_datastore="20newsgroups/20news.pkl")
print("DataReference object created")

### Intermediate/Output Data
Intermediate data (or output of a Step) is represented by **[PipelineData](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipelinedata?view=azure-ml-py)** object. PipelineData can be produced by one step and consumed in another step by providing the PipelineData object as an output of one step and the input of one or more steps.

#### Constructing PipelineData
- **name:** [*Required*] Name of the data item within the pipeline graph
- **datastore_name:** Name of the Datastore to write this output to
- **output_name:** Name of the output
- **output_mode:** Specifies "upload" or "mount" modes for producing output (default: mount)
- **output_path_on_compute:** For "upload" mode, the path to which the module writes this output during execution
- **output_overwrite:** Flag to overwrite pre-existing data

In [ ]:
# Define intermediate data using PipelineData
# Syntax
# PipelineData(name, 
#              datastore_name=None, 
#              output_name=None, 
#              output_mode='mount', 
#              output_path_on_compute=None, 
#              output_overwrite=None)

# Naming the intermediate data as processed_data1 and assigning it to the variable processed_data1.
processed_data1 = PipelineData("processed_data1",datastore=def_blob_s)
print("PipelineData object created")

### Pipelines steps using datasources and intermediate data
Machine learning pipelines have many steps and these steps could use or reuse datasources and intermediate data. Here's how we construct such a pipeline:

#### Define a Step that consumes a datasource and produces intermediate data.
In this step, we define a step that consumes a datasource and produces intermediate data.

**Open `train.py` in the local machine and examine the arguments, inputs, and outputs for the script. That will give you a good sense of why the script argument names used below are important.** 

In [ ]:
# step4 consumes the datasource (Datareference) in the previous step
# and produces processed_data1
trainStep = PythonScriptStep(
    script_name="train.py", 
    arguments=["--input_data", blob_input_data, "--output_train", processed_data1],
    inputs=[blob_input_data],
    outputs=[processed_data1],
    target=batch_ai, 
    source_directory=project_folder
)
print("trainStep created")

#### Define a Step that consumes intermediate data and produces intermediate data
In this step, we define a step that consumes an intermediate data and produces intermediate data.

**Open `extract.py` in the local machine and examine the arguments, inputs, and outputs for the script. That will give you a good sense of why the script argument names used below are important.** 

In [ ]:
# step5 to use the intermediate data produced by step4
# This step also produces an output processed_data2
processed_data2 = PipelineData("processed_data2", datastore=def_blob_s)

extractStep = PythonScriptStep(
    script_name="extract.py",
    arguments=["--input_extract", processed_data1, "--output_extract", processed_data2],
    inputs=[processed_data1],
    outputs=[processed_data2],
    target=batch_ai, 
    source_directory=project_folder)
print("extractStep created")

#### Define a Step that consumes multiple intermediate data and produces intermediate data
In this step, we define a step that consumes multiple intermediate data and produces intermediate data.

**Open `compare.py` in the local machine and examine the arguments, inputs, and outputs for the script. That will give you a good sense of why the script argument names used below are important.**

In [ ]:
# Now define step6 that takes two inputs (both intermediate data), and produce an output
processed_data3 = PipelineData("processed_data3", datastore=def_blob_s)

compareStep = PythonScriptStep(
    script_name="compare.py",
    arguments=["--compare_data1", processed_data1, "--compare_data2", processed_data2, "--output_compare", processed_data3],
    inputs=[processed_data1, processed_data2],
    outputs=[processed_data3],    
    target=batch_ai, 
    source_directory=project_folder)
print("compareStep created")

#### Build the pipeline

In [ ]:
pipeline1 = Pipeline(workspace=ws, steps=[compareStep])
print ("Pipeline is built")

pipeline1.validate()
print("Simple validation complete") 

In [ ]:
pipeline_run1 = Experiment(ws, 'Data_dependency').submit(pipeline1)
print("Pipeline is submitted for execution")

In [ ]:
RunDetails(pipeline_run1).show()

# Next: Publishing the Pipeline and calling it from the REST endpoint
See this [notebook](./03.publish-pipeline-and-run-using-rest-endpoint.ipynb) to understand how the pipeline is published and you can call the REST endpoint to run the pipeline.